# PyTorch: Variable e Gradientes e Grafo Computacional

Um dos principais fundamentos para que o PyTorch seja adequado para deep learning é a sua habilidade de
calcular o gradiente automaticamente a partir da expressões definidas. Essa facilidade é implementada
pelo tipo Variable do PyTorch, que adiciona ao tensor a facilidade de cálculo automático do gradiente pela construção dinâmica do grafo computacional.

<img src='../figures/GrafoComputacional.png', width=600pt></img>

Variable possui 3 campos: o dado em si (data), o gradiente (grad) e um apontador (creator) para construir o grafo da backpropagation. Uma expressão utilizada para o cálculo do gradiente exige que todas suas expressões sejam calculadas com Variables, caso contrário não é possível construir o grafo computacional.

<img src='../figures/variables.png'></a>

In [3]:
import torch
from torch.autograd import Variable

## Variable é criada a partir de um tensor

In [4]:
y_t = 2 * torch.arange(0,4)
y = Variable(y_t); y

Variable containing:
 0
 2
 4
 6
[torch.FloatTensor of size 4]

In [5]:
x = Variable(torch.arange(0,4)); x

Variable containing:
 0
 1
 2
 3
[torch.FloatTensor of size 4]

In [6]:
w = Variable(torch.ones(1),requires_grad=True); w

Variable containing:
 1
[torch.FloatTensor of size 1]

## Variable possui as mesmas funcionalidades dos tensores

## Cálculo automático do gradiente da função perda J

Seja a expressão: $$ J = ((x  w) - y)^2 $$

Queremos calcular a derivada de $J$ em relação a $w$.

### Montagem do grafo computacional

In [7]:
# predict (forward)
y_pred = x * w

# cálculo da perda J: loss
e = y_pred - y
e2 = e.pow(2)
J = (e2).sum()
J

Variable containing:
 14
[torch.FloatTensor of size 1]

## Auto grad - processa o grafo computacional backwards

O `backward()` varre o grafo computacional a partir da variável a ele associada e calcula o gradiente para todas as `Variables` que possuem o atributo `requires_grad` como verdadeiro.
O `backward()` destroi o grafo após sua execução. Isso é intrínsico ao PyTorch pelo fato dele ser uma rede dinâmica.

In [8]:
J.backward()
print(w.grad)

Variable containing:
-28
[torch.FloatTensor of size 1]



In [9]:
w.grad.data.zero_();

# Interpretação do Gradiente

O gradiente de uma variável final (J) com respeito à outra variável de entrada (w) pode ser interpretado como o quanto a variável final J vai aumentar se houver um pequeno aumento na variável de entrada (w).
Por exemplo suponha que o gradiente seja 28. Isto significa se aumentarmos a variável w de 0.001, então J vai aumentar de 0.028.

In [13]:
eps = 0.001
y_pred = x * (w+eps)
J = (y_pred - y).pow(2).sum()
J

Variable containing:
 13.9720
[torch.FloatTensor of size 1]

## Back propagation

Uma forma equivalente explícita de calcular o gradiente é fazendo o processamento do backpropagation no grafo computacional, de forma explícita.
Apenas como ilustração.

In [12]:
import numpy as np

dJ = 1.
de2 = dJ * np.ones((4,))
de = de2 * 2 * e.data.numpy()
dy_pred = de
dw = (dy_pred * x.data.numpy()).sum()
print(dJ)
print(de2)
print(de)
print(dw)


1.0
[ 1.  1.  1.  1.]
[ 0. -2. -4. -6.]
-28.0


# Exercícios

1. Atualize um passo de atualização do valor de w, pelo 
gradiente descendente. Utilize um fator de aprendizado (*learning rate*) de 0.1 
e atualize o w e recalcule a função de perda novamente:

    - w = w - lr * w.grad.data
    - execute a célula 1.3.1 e verifique o quanto que a perda J diminuiu